In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("demo").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/11 12:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
itemsSchema = ("id integer, name string, price float")
ordersSchema=("id integer, itemid integer, count integer")
items = spark.createDataFrame([[0, "Tomato", 2.0], \
                            [1, "Watermelon", 5.5], \
                            [2, "pineapple", 7.0]], \
                            schema=itemsSchema)
orders = spark.createDataFrame([[100, 0, 1], \
                            [100, 1, 1], \
                            [101, 2, 3], \
                            [102,2,8]],\
                            schema=ordersSchema)

In [20]:
from pyspark.sql.functions import sum
y=(items.join(orders,items.id==orders.itemid, how="inner"))\
        .where(items.id==2)\
        .groupBy("name","price").agg(sum("count")\
        .alias("c"))

In [22]:
y.explain(mode="extended")

y.explain(mode="formatted")

y.explain(mode="codegen")

y.explain(mode="cost")

== Parsed Logical Plan ==
'Aggregate ['name, 'price], ['name, 'price, 'sum('count) AS c#150]
+- Filter (id#136 = 2)
   +- Join Inner, (id#136 = itemid#140)
      :- LogicalRDD [id#136, name#137, price#138], false
      +- LogicalRDD [id#139, itemid#140, count#141], false

== Analyzed Logical Plan ==
name: string, price: float, c: bigint
Aggregate [name#137, price#138], [name#137, price#138, sum(count#141) AS c#150L]
+- Filter (id#136 = 2)
   +- Join Inner, (id#136 = itemid#140)
      :- LogicalRDD [id#136, name#137, price#138], false
      +- LogicalRDD [id#139, itemid#140, count#141], false

== Optimized Logical Plan ==
Aggregate [name#137, price#138], [name#137, price#138, sum(count#141) AS c#150L]
+- Project [name#137, price#138, count#141]
   +- Join Inner, (id#136 = itemid#140)
      :- Filter (isnotnull(id#136) AND (id#136 = 2))
      :  +- LogicalRDD [id#136, name#137, price#138], false
      +- Project [itemid#140, count#141]
         +- Filter ((itemid#140 = 2) AND isnotnull(i